<a href="https://colab.research.google.com/github/Lucia1009/TFG-Plataforma-para-la-prediccion-de-potencia-electrica-generada-en-parques-eolicos/blob/desarrollo/modelos/comp_datos_modelos_santiago.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openmeteo-requests -q
!pip install requests-cache retry-requests -q

!pip install dash -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 10.3 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np
import math

import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error

rosa='\033[95m'
fin='\033[0m'

# Datos página

In [3]:
"""Histórico de datos de la página"""

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 42.87596,
	"longitude": -8.559434,
	"start_date": "2023-01-01",
	"end_date": "2024-12-30",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m"],
	"models": ["best_match", "era5_seamless", "era5", "era5_land", "cerra", "ecmwf_ifs", "ecmwf_ifs_analysis_long_window", "era5_ensemble"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

							# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"Fecha": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperatura"] = hourly_temperature_2m
hourly_data["wind_speed"] = hourly_wind_speed_10m
hourly_data["wind_direction"] = hourly_wind_direction_10m

hist_pag = pd.DataFrame(data = hourly_data)
hist_pag.dropna(inplace = True)
hist_pag['Fecha'] = pd.to_datetime(hist_pag['Fecha'])
hist_pag['Fecha'] = hist_pag['Fecha'].dt.tz_localize(None)
hist_pag.set_index('Fecha', inplace=True)

display(hist_pag)

Coordinates 42.84709930419922°N -8.5333251953125°E
Elevation 239.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s


,temperatura,wind_speed,wind_direction
Fecha,,,
2023-01-01 00:00:00,10.4925,14.618837,189.926178
2023-01-01 01:00:00,10.6425,12.605142,178.363464
2023-01-01 02:00:00,10.9925,12.074766,169.695206
2023-01-01 03:00:00,11.0425,10.137692,163.495560
2023-01-01 04:00:00,11.0425,7.200000,126.869987
...,...,...,...
2024-12-30 19:00:00,5.6925,2.753470,168.690109
2024-12-30 20:00:00,5.2925,1.484318,165.963730
2024-12-30 21:00:00,4.5425,0.917823,101.309898


In [4]:
"""Predicciones históricas"""

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://historical-forecast-api.open-meteo.com/v1/forecast"

lista_modelos=["ecmwf_ifs04", "gfs_global", "jma_gsm", "icon_global", "icon_eu", "icon_d2", "gem_global", "ukmo_global_deterministic_10km", "meteofrance_arpege_europe", "meteofrance_arome_france_hd"]

params = {
	"latitude": 42.87596,
	"longitude": -8.559434,
	"start_date": "2023-01-01",
	"end_date": "2024-12-30",
	"hourly": ["temperature_2m", "wind_speed_10m", "wind_direction_10m"],
	"models": lista_modelos
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
models=dict()
for response, mod in zip(responses, lista_modelos):

  print(f"{rosa}Model {mod}{fin}")  #Imprime con colores
  print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
  print(f"Elevation {response.Elevation()} m asl")
  print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
  print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s\n")


  # Process hourly data. The order of variables needs to be the same as requested.
  hourly = response.Hourly()
  hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
  hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
  hourly_wind_direction_10m = hourly.Variables(2).ValuesAsNumpy()

  hourly_data = {"Fecha": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
  )}

  hourly_data["temperatura"] = hourly_temperature_2m
  hourly_data["wind_speed"] = hourly_wind_speed_10m
  hourly_data["wind_direction"] = hourly_wind_direction_10m

  hourly_dataframe = pd.DataFrame(data = hourly_data)
  hourly_dataframe.dropna(inplace = True)

  if not hourly_dataframe.empty:
    hourly_dataframe['wd_sin_10m'] = np.sin(hourly_dataframe['wind_direction']*np.pi/180)
    hourly_dataframe['wd_cos_10m'] = np.cos(hourly_dataframe['wind_direction']*np.pi/180)

    models[mod]=hourly_dataframe
    models[mod]['Fecha']= models[mod]['Fecha'].dt.tz_localize(None)
    models[mod].set_index('Fecha', inplace=True)
    print(hourly_dataframe.iloc[0], '\n\n')
  else:
    if mod in lista_modelos:
            lista_modelos.remove(mod)
    print(f"Warning: DataFrame for model {mod} is empty after dropping NaNs.\n\n")

print(len(models))
print(models.keys())

Model ecmwf_ifs04
Coordinates 42.80000305175781°N -8.399993896484375°E
Elevation 239.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s

temperatura        11.050000
wind_speed         14.917212
wind_direction    188.325562
wd_sin_10m         -0.144798
wd_cos_10m         -0.989461
Name: 2023-01-01 00:00:00, dtype: float32 


Model gfs_global
Coordinates 42.81809997558594°N -8.5546875°E
Elevation 239.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s

temperatura        10.694000
wind_speed          7.952660
wind_direction    185.194336
wd_sin_10m         -0.090534
wd_cos_10m         -0.995893
Name: 2023-01-01 00:00:00, dtype: float32 


Model jma_gsm
Coordinates 43.0°N -8.5°E
Elevation 239.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s

temperatura         9.950000
wind_speed         10.464798
wind_direction    176.054886
wd_sin_10m          0.068801
wd_cos_10m         -0.997630
Name: 2023-01-01 00:00:00, dtype: float32 


Model icon_global
Coordinates 42

# Datos reales

In [12]:
# Abro los datos del json y lo guardo en un dataframe

df = pd.read_json('/content/RedMeteorológica_Santiago-EOAS_diezminutal_01-01-2023_to_01-01-2024.json')

# display(df)

# Hago un nuevo dataframe que tenga como columnas las variables de la columna variable y el código de validación, los valores de la columna valor y del código de validación y el índice de la columna fecha
unique_codigos = df['Código'].unique().tolist()
datos = pd.DataFrame(columns=['Fecha']+unique_codigos )

for codigo in unique_codigos:

  variable= df[df['Código'] ==codigo][['Fecha','Código','Valor', 'Código de validación']]
  variable=variable[variable['Código de validación']==1]

  datos['Fecha'] = df['Fecha'].unique()

  merged_data = datos.merge(variable[['Fecha', 'Valor']], on='Fecha', how='left')
  datos[codigo] = merged_data['Valor']

datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y %H:%M')
datos['Fecha']=datos['Fecha'].dt.tz_localize(None)
datos.set_index('Fecha', inplace=True)


# display(datos.head(10))
print(datos.shape)

# Datos con la media de cada hora
datos_hora=datos.resample('h').mean()

#renombramos columnas
datos_hora = datos_hora.rename(columns={'TA_AVG_1.5m': 'temperatura', 'VV_AVG_10m': 'wind_speed', 'DV_AVG_10m': 'wind_direction'})


# display(datos_hora.head(10))
print(datos_hora.shape)

(52698, 3)
(8785, 3)


# Obtener métricas para el histograma de comparación de modelos

In [13]:
# Obtengo el error de la dirección a través del error del seno y el coseno

def diferencia_angulos(sin_real, cos_real, sin_pagina, cos_pagina):

  producto_escalar = sin_real * sin_pagina + cos_real * cos_pagina
  producto_escalar = max(-1.0, min(1.0, producto_escalar))
  angulo_rad = math.acos(producto_escalar)
  angulo_deg = math.degrees(angulo_rad)
  return angulo_deg


In [14]:
atributos=['temperatura', 'wind_speed']

def obtener_metricas(historico_df, models):

  scores=dict()
  error_angulos=dict()

  for mod in models.keys():

    scores[mod]=dict()

    hist_df=historico_df.copy()
    # hist_df = hist_df.reset_index()

    for atributo in atributos:

      datos_trabajo_model=hist_df.merge(models[mod], on="Fecha", how="inner", suffixes=("", f"_{mod}"),)
      datos_trabajo_model.dropna(inplace=True)

      y_true=datos_trabajo_model[atributo]
      y_pred_model=datos_trabajo_model[atributo+f"_{mod}"]
      rmse = np.sqrt(mean_squared_error(y_true, y_pred_model))
      mae = mean_absolute_error(y_true, y_pred_model)
      r2 = r2_score(y_true, y_pred_model)
      mx_error = max_error(y_true, y_pred_model)


      scores[mod][atributo]={'rmse':rmse, 'mae':mae, 'r2':r2, 'max_error':mx_error}

    sin_y_true=np.sin(datos_trabajo_model['wind_direction']*np.pi/180)
    cos_y_true=np.cos(datos_trabajo_model['wind_direction']*np.pi/180)
    sin_y_pred_model=np.sin(datos_trabajo_model[f'wind_direction_{mod}']*np.pi/180)
    cos_y_pred_model=np.cos(datos_trabajo_model[f'wind_direction_{mod}']*np.pi/180)


    error_angulos[mod]=np.mean([diferencia_angulos(sin_y_true.iloc[k], cos_y_true.iloc[k], sin_y_pred_model.iloc[k], cos_y_pred_model.iloc[k]) for k in range(len(sin_y_true))])


  return scores, error_angulos

scores, error_angulos=obtener_metricas(hist_pag, models)

for mod in models.keys():
    print(f"\n{rosa}Model {mod}{fin}")
    for atributo in atributos:
        print(f"{atributo}: ", end="") #Imprime sin salto de línea
        for metric, value in scores[mod][atributo].items():
            print(f"{metric}: {value} ", end="")
        print()
    print(f"Error de dirección medio: {error_angulos[mod]}")



Model ecmwf_ifs04
temperatura: rmse: 0.8181320639495481 mae: 0.589694082736969 r2: 0.9792953729629517 max_error: 4.857500076293945 
wind_speed: rmse: 2.54988650275074 mae: 1.9046121835708618 r2: 0.8847042322158813 max_error: 18.485612869262695 
Error de dirección medio: 19.974255903427455

Model gfs_global
temperatura: rmse: 1.6595205009736564 mae: 1.2766273021697998 r2: 0.9148106575012207 max_error: 9.59850025177002 
wind_speed: rmse: 6.743947176308968 mae: 5.511405944824219 r2: 0.19350790977478027 max_error: 28.095474243164062 
Error de dirección medio: 20.51167099313406

Model jma_gsm
temperatura: rmse: 1.4270462903786347 mae: 1.0897815227508545 r2: 0.9370064735412598 max_error: 7.242500305175781 
wind_speed: rmse: 6.190343646492488 mae: 5.029831886291504 r2: 0.32048141956329346 max_error: 24.3649959564209 
Error de dirección medio: 26.55168110187237

Model icon_global
temperatura: rmse: 0.9826617393637546 mae: 0.7417620420455933 r2: 0.9701305031776428 max_error: 6.253999710083008 



---


# Comparaciones


In [15]:
"""Gráfica de datos para cada modelo y variable"""

atribs=['temperatura', 'wind_speed', 'wind_direction']

app = Dash('app1')

app.layout = html.Div([

  dcc.Dropdown(lista_modelos, lista_modelos[0], id='modelo', clearable=False),
  dcc.Dropdown(atribs, atribs[0], id='atributo', clearable=False),
  dcc.Checklist(
          id='ver_error',
          options=[{'label': 'Cambiar a visualizar error', 'value': 'error'}],
          value=[]
  ),
  dcc.Checklist(
          id='ver_pagina',
          options=[{'label': 'Ver los datos historicos de la pagina', 'value': 'pag'}],
          value=['pag']
  ),
  dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('modelo', 'value'),
    Input('atributo', 'value'),
    Input('ver_error', 'value'),
    Input('ver_pagina', 'value'))
def update_output_div(modelo, atributo, ver_error, ver_pagina):

  fig = go.Figure()


  datos_trabajo_model=hist_pag.merge(models[modelo], on="Fecha", how="inner", suffixes=("", f"_{modelo}"),)


  datos_trabajo_est=datos_hora.merge(datos_trabajo_model, on="Fecha", how="inner", suffixes=("", f"_est"),)
  datos_trabajo_est.dropna(inplace=True)

  y_true=datos_trabajo_est[atributo]
  y_pred_model=datos_trabajo_est[atributo+f"_{modelo}"]
  y_true_est=datos_trabajo_est[atributo+f"_est"]


  if 'error' in ver_error:
    if atributo != 'wind_direction':
      if 'pag' in ver_pagina:
        fig.add_trace(go.Scatter(y=y_true-y_pred_model, mode='lines', marker=dict(color='red'), name='Error pagina'))
      fig.add_trace(go.Scatter(y=y_true_est-y_pred_model, mode='lines', opacity=0.5, marker=dict(color='blue'), name='Error estación'))
      fig.update_layout(
          title='Error historico vs prediccion',
          transition_duration=10,
          xaxis_title='tiempo (h)',
          yaxis_title=f'Error{atributo}',
          showlegend=True
      )
    else:
      print(datos_trabajo_est.columns)
      dir_true=datos_trabajo_est['wind_direction']
      dir_pred=datos_trabajo_est['wind_direction'+f"_{modelo}"]
      dir_true_est=datos_trabajo_est['wind_direction'+f"_est"]

      sin_y_true=np.sin(dir_true*np.pi/180)
      cos_y_true=np.cos(dir_true*np.pi/180)
      sin_y_pred_model=np.sin(dir_pred*np.pi/180)
      cos_y_pred_model=np.cos(dir_pred*np.pi/180)
      sin_y_true_est=np.sin(dir_true_est*np.pi/180)
      cos_y_true_est=np.cos(dir_true_est*np.pi/180)

      if 'pag' in ver_pagina:
        error_pag=[diferencia_angulos(sin_y_true.iloc[k], cos_y_true.iloc[k], sin_y_pred_model.iloc[k], cos_y_pred_model.iloc[k]) for k in range(len(y_true))]
        fig.add_trace(go.Scatter(y=error_pag, mode='lines', marker=dict(color='red'), name='Error ángulo pag'))

      error_est=[diferencia_angulos(sin_y_true_est.iloc[k], cos_y_true_est.iloc[k], sin_y_pred_model.iloc[k], cos_y_pred_model.iloc[k]) for k in range(len(y_true))]
      fig.add_trace(go.Scatter(y=error_est, mode='lines', opacity=0.5, marker=dict(color='blue'), name='Error ángulo est'))

  else:
    fig.add_trace(go.Scatter(y=y_pred_model, mode='lines', marker=dict(color='red'), name='Predichas'))
    fig.add_trace(go.Scatter(y=y_true_est, mode='lines', marker=dict(color='blue'), opacity=0.5, name='Historicas est'))

    if 'pag' in ver_pagina:
      fig.add_trace(go.Scatter(y=y_true, mode='lines', marker=dict(color='green'), opacity=0.5, name='Historicas pag'))

    fig.update_layout(
        title='Historico vs prediccion',
        transition_duration=10,
        xaxis_title='tiempo (h)',
        yaxis_title=f'{atributo}',
        showlegend=True
    )



  return fig



if __name__ == '__main__':
    app.run(debug=True, port=8071)


<IPython.core.display.Javascript object>

In [27]:
seleccionados = ['hist_pag', 'datos_hora']

app = Dash('app1')

app.layout = html.Div([
    dcc.Dropdown(seleccionados, seleccionados[1], id='seleccion', clearable=False),
    dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('seleccion', 'value'))
def update_output_div(seleccion):
    if seleccion == 'hist_pag':
        datos = hist_pag
    else:
        datos = datos_hora

    scores, error_angulos = obtener_metricas(datos, models)

    lista_modelos = list(scores.keys())

    # Define las métricas
    metrics = ['rmse', 'mae', 'r2', 'max_error']

    # Importa las librerías de Plotly necesarias
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    import plotly.express as px

    # Crea subplots: una fila por cada métrica
    fig = make_subplots(
        rows=len(metrics), cols=1,
        subplot_titles=[f'{metric.upper()} Scores per Variable and Model' for metric in metrics],
        shared_xaxes=True
    )

    # Asigna colores a cada modelo utilizando la escala Viridis de Plotly
    model_colors = px.colors.sample_colorscale(
        "Viridis",
        [i / (len(lista_modelos) - 1) if len(lista_modelos) > 1 else 0.5 for i in range(len(lista_modelos))]
    )

    # Para cada métrica (cada subplot)
    for metric_index, metric in enumerate(metrics):
        row_number = metric_index + 1

        # Para cada modelo, se construyen los valores de y para cada variable
        for model_index, model in enumerate(lista_modelos):
            y_values = []
            for variable in atribs:
                if variable == 'wind_direction':
                  if metric=='rmse':
                    score = error_angulos[model]
                  else:
                    score = float('nan')
                else:
                    score = scores[model][variable][metric]
                y_values.append(score)


            # Agrega la traza de barras para este modelo y métrica
            fig.add_trace(
                go.Bar(
                    x=atribs,
                    y=y_values,
                    name=model,
                    marker_color=model_colors[model_index],
                    text=[f'{val:.2f}' for val in y_values],
                    textposition='auto'
                ),
                row=row_number,
                col=1
            )

        # Configura el título del eje y para el subplot correspondiente
        fig.update_yaxes(title_text=metric.upper(), row=row_number, col=1)

    # Configura el título del eje x para el último subplot
    fig.update_xaxes(title_text="Variables", row=len(metrics), col=1)

    # Actualiza la configuración general del layout
    fig.update_layout(
        barmode='group',
        height=1000,
        width=1200,
        legend_title_text='Modelos'
    )

    return fig


if __name__ == '__main__':
    app.run(debug=True, port=8072)


<IPython.core.display.Javascript object>

In [29]:
"""Histograma de 80% y 90% de error"""

# Histograma con el número de incidencias de cada error

"""Gráfica de datos para cada modelo y variable"""

app = Dash('app1')

app.layout = html.Div([

  dcc.Dropdown(lista_modelos, lista_modelos[0], id='modelo', clearable=False),
  dcc.Dropdown(atribs, atribs[0], id='atributo', clearable=False),
  dcc.Checklist(
          id='ver_pagina',
          options=[{'label': 'Histograma de los datos historicos de la pagina', 'value': 'pag'}],
          value=[]
  ),
  dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('modelo', 'value'),
    Input('atributo', 'value'),
    Input('ver_pagina', 'value'))
def update_output_div(modelo, atributo, ver_pagina):


  fig = go.Figure()

  datos_trabajo_model=hist_pag.merge(models[modelo], on="Fecha", how="inner", suffixes=("", f"_{modelo}"),)


  datos_trabajo_est=datos_hora.merge(datos_trabajo_model, on="Fecha", how="inner", suffixes=("", f"_est"),)
  datos_trabajo_est.dropna(inplace=True)

  if 'pag' in ver_pagina:
    y_true=datos_trabajo_est[atributo]
    y_pred_model=datos_trabajo_est[atributo+f"_{modelo}"]


    if atributo != 'wind_direction':
      residuals = y_true -y_pred_model
    else:

      dir_true=datos_trabajo_est['wind_direction']
      dir_pred=datos_trabajo_est['wind_direction'+f"_{modelo}"]


      sin_y_true=np.sin(dir_true*np.pi/180)
      cos_y_true=np.cos(dir_true*np.pi/180)
      sin_y_pred_model=np.sin(dir_pred*np.pi/180)
      cos_y_pred_model=np.cos(dir_pred*np.pi/180)

      residuals=[diferencia_angulos(sin_y_true.iloc[k], cos_y_true.iloc[k], sin_y_pred_model.iloc[k], cos_y_pred_model.iloc[k]) for k in range(len(y_true))]

        # Create the interactive histogram
    fig = px.histogram(
        x=residuals,
        # nbins=800,  # Adjust the number of bins as needed
        title="Histograma de Residuos de predicciones históricas con el histórico de la página",
        labels={"x": "Residuos", "y": "Frecuencia"},
    )

  else:

    y_pred_model=datos_trabajo_est[atributo+f"_{modelo}"]
    y_true_est=datos_trabajo_est[atributo+f"_est"]


    if atributo != 'wind_direction':
      residuals = y_true_est -y_pred_model
    else:

      dir_pred=datos_trabajo_est['wind_direction'+f"_{modelo}"]
      dir_true_est=datos_trabajo_est['wind_direction'+f"_est"]

      sin_y_pred_model=np.sin(dir_pred*np.pi/180)
      cos_y_pred_model=np.cos(dir_pred*np.pi/180)
      sin_y_true_est=np.sin(dir_true_est*np.pi/180)
      cos_y_true_est=np.cos(dir_true_est*np.pi/180)

      residuals=[diferencia_angulos(sin_y_true_est.iloc[k], cos_y_true_est.iloc[k], sin_y_pred_model.iloc[k], cos_y_pred_model.iloc[k]) for k in range(len(y_true_est))]

    # Create the interactive histogram
    fig = px.histogram(
        x=residuals,
        # nbins=800,  # Adjust the number of bins as needed
        title="Histograma de Residuos de predicciones históricas con los datos históricos de la estación",
        labels={"x": "Residuos", "y": "Frecuencia"},
    )


  # Calcula el percentil 5 y 95
  percentil_5 = np.percentile(residuals, 5)
  percentil_95 = np.percentile(residuals, 95)

  # Agrega las líneas verticales
  fig.add_vline(x=percentil_5, line_width=3, line_dash="dash", name="Percentil 5", line_color="red")
  fig.add_vline(x=percentil_95, line_width=3, line_dash="dash", line_color="red")

  # Calcula el percentil 5 y 95
  percentil_10 = np.percentile(residuals, 10)
  percentil_90 = np.percentile(residuals, 90)

  # Agrega las líneas verticales
  fig.add_vline(x=percentil_10, line_width=3, line_dash="dash", line_color="green")
  fig.add_vline(x=percentil_90, line_width=3, line_dash="dash", line_color="green")

  return fig



if __name__ == '__main__':
    app.run(debug=True, port=8073)


<IPython.core.display.Javascript object>

In [34]:
"""Histograma de 80% y 90% de error"""

# Histograma con el número de incidencias de cada error

"""Gráfica de datos para cada modelo y variable"""

app = Dash('app1')

app.layout = html.Div([

  dcc.Dropdown(atribs, atribs[0], id='atributo', clearable=False),
  dcc.Graph(id='graph')
])


@app.callback(
    Output('graph', 'figure'),
    Input('atributo', 'value'))
def update_output_div(atributo):


  fig = go.Figure()

  datos_trabajo_est=datos_hora.merge(hist_pag, on="Fecha", how="inner", suffixes=("", f"_est"),)
  datos_trabajo_est.dropna(inplace=True)

  y_true_pag=datos_trabajo_est[atributo]
  y_true_est=datos_trabajo_est[atributo+f"_est"]


  if atributo != 'wind_direction':
    residuals = y_true_pag -y_true_est
  else:

    dir_true_pag=datos_trabajo_est['wind_direction']
    dir_true_est=datos_trabajo_est['wind_direction'+f"_est"]


    sin_y_true_pag=np.sin(dir_true_pag*np.pi/180)
    cos_y_true_pag=np.cos(dir_true_pag*np.pi/180)
    sin_y_true_est=np.sin(dir_true_est*np.pi/180)
    cos_y_true_est=np.cos(dir_true_est*np.pi/180)

    residuals=[diferencia_angulos(sin_y_true_pag.iloc[k], sin_y_true_pag.iloc[k], sin_y_true_est.iloc[k], sin_y_true_est.iloc[k]) for k in range(len(y_true_pag))]

      # Create the interactive histogram
  fig = px.histogram(
      x=residuals,
      # nbins=800,  # Adjust the number of bins as needed
      title="Histograma de Residuos del histórico real de la estación con el histórico de la página",
      labels={"x": "Residuos", "y": "Frecuencia"},
  )


  # Calcula el percentil 5 y 95
  percentil_5 = np.percentile(residuals, 5)
  percentil_95 = np.percentile(residuals, 95)

  # Agrega las líneas verticales
  fig.add_vline(x=percentil_5, line_width=3, line_dash="dash", name="Percentil 5", line_color="red")
  fig.add_vline(x=percentil_95, line_width=3, line_dash="dash", line_color="red")

  # Calcula el percentil 5 y 95
  percentil_10 = np.percentile(residuals, 10)
  percentil_90 = np.percentile(residuals, 90)

  # Agrega las líneas verticales
  fig.add_vline(x=percentil_10, line_width=3, line_dash="dash", line_color="green")
  fig.add_vline(x=percentil_90, line_width=3, line_dash="dash", line_color="green")

  return fig



if __name__ == '__main__':
    app.run(debug=True, port=8073)


<IPython.core.display.Javascript object>